In [2]:
from pyspark.sql import SparkSession, Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import RandomForestClassificationModel

# Khởi tạo Spark session
spark = SparkSession.builder.appName("PredictWithSavedModel").getOrCreate()

# Load mô hình đã lưu
model = RandomForestClassificationModel.load("./random_forest_model")

# Nhiều bộ dữ liệu test
test_inputs = [
    [42.5, 70.1, 0.7, 18.3, 9.2, 25.0, 29.4, 65.0, 0.0, 1.5],
    [15.2, 30.0, 0.3, 8.0, 3.0, 20.0, 26.0, 60.0, 0.0, 1.0],
    [85.0, 150.0, 1.2, 40.0, 20.0, 35.0, 35.0, 80.0, 0.0, 2.5],
    [60.0, 80.0, 0.5, 20.0, 10.0, 28.0, 30.0, 70.0, 0.1, 1.2],
    [5.0, 10.0, 0.1, 3.0, 1.0, 10.0, 24.0, 50.0, 0.0, 0.5]
]

# Chuyển thành DataFrame
rows = [Row(Vectors.dense(values)) for values in test_inputs]
input_df = spark.createDataFrame(rows, ["features"])

# Dự đoán
predictions = model.transform(input_df)

# Mapping nhãn
label_mapping = {
    0: "Kém",
    1: "Rất Kém",
    2: "Trung bình",
    3: "Tốt"
}

# In kết quả cho từng dòng
print("\n===== KẾT QUẢ DỰ ĐOÁN NHIỀU DÒNG =====")
for i, row in enumerate(predictions.select("features", "prediction", "probability").collect()):
    pred_label = int(row["prediction"])
    prob = row["probability"]
    status_icon = "🟢" if pred_label == 3 else "🔴"
    print(f"\n▶️ Dòng {i+1}:")
    print(f"  Dữ liệu đầu vào  : {row['features']}")
    print(f"  Dự đoán chất lượng: {label_mapping[pred_label]} {status_icon}")
    print(f"  Xác suất dự đoán : {prob}")



===== KẾT QUẢ DỰ ĐOÁN NHIỀU DÒNG =====

▶️ Dòng 1:
  Dữ liệu đầu vào  : [42.5,70.1,0.7,18.3,9.2,25.0,29.4,65.0,0.0,1.5]
  Dự đoán chất lượng: Trung bình 🔴
  Xác suất dự đoán : [0.1664623385359979,0.03504890316902429,0.5626631247844454,0.23582563351053248]

▶️ Dòng 2:
  Dữ liệu đầu vào  : [15.2,30.0,0.3,8.0,3.0,20.0,26.0,60.0,0.0,1.0]
  Dự đoán chất lượng: Tốt 🟢
  Xác suất dự đoán : [0.17393309180874209,0.05729466322937467,0.2619917558149502,0.5067804891469331]

▶️ Dòng 3:
  Dữ liệu đầu vào  : [85.0,150.0,1.2,40.0,20.0,35.0,35.0,80.0,0.0,2.5]
  Dự đoán chất lượng: Kém 🔴
  Xác suất dự đoán : [0.8457553434888953,0.01998897643248606,0.08029461765932307,0.05396106241929555]

▶️ Dòng 4:
  Dữ liệu đầu vào  : [60.0,80.0,0.5,20.0,10.0,28.0,30.0,70.0,0.1,1.2]
  Dự đoán chất lượng: Kém 🔴
  Xác suất dự đoán : [0.8343717919598947,0.018646415189343332,0.08678246923346454,0.06019932361729728]

▶️ Dòng 5:
  Dữ liệu đầu vào  : [5.0,10.0,0.1,3.0,1.0,10.0,24.0,50.0,0.0,0.5]
  Dự đoán chất lượng: Tốt 🟢
 